In [59]:
# Imports
import tensorflow as tf
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import time
import csv
import os

# Some helper functions
def accuracy(predictions, labels):
    return np.linalg.norm(predictions - labels)

# Let's define a simple NN
class NeuralNet:
    ########### PRIVATE METHODS ###################
    # Geneates a fully connected model from the input, weights and biases
    # keeping the number of layers and the dropout probability in mind
    def __genModel(self, x, w, b, num_layers, keep_prob):
        # Local variable activations
        a = dict()
        
        if num_layers == 0:
            return tf.nn.bias_add(tf.matmul(x, w[0]), b[0])
        else:
            a[0] = tf.nn.dropout(tf.nn.relu(tf.nn.bias_add(tf.matmul(x, w[0]), b[0])), keep_prob)
            for i in np.arange(1, num_layers):
                a[i] = tf.nn.dropout(tf.nn.relu(tf.nn.bias_add(tf.matmul(a[i-1], w[i]), b[i])), keep_prob)
            i = num_layers
            return tf.nn.bias_add(tf.matmul(a[i-1], w[i]), b[i])
        
    ########### PUBLIC METHODS ###################
    # Constructor
    def __init__(self, batch_size, x_size, y_size):
        # Store the values
        self.batch_size = batch_size
        self.x_size = x_size
        self.y_size = y_size
        
        # Initialize the graph
        self.graph = tf.Graph()
    
    """
    Adds a configurable number of layers into the graph
    num_layers: Number of fully connected layers to be added
    w_sd: The allowed std-dev of the weights
    num_nodes: A np array containing the number of nodes in each layer,
        so num_nodes must be a list of num_layers length
    keep_prob: The dropouts keep prob
    reg_const: The L2 norm constant
    init_lrate: Initial learning rate
    lr_decay_step: Number of steps after which learning rate should decay
    lr_decay_rate: The rate at which the learning rate should decay
    """
    def Model(self, num_layers, w_sd, num_nodes, keep_prob, reg_const, init_lrate, lr_decay_step, lr_decay_rate):
        # Preconditions
        assert num_layers == num_nodes.shape[0]
        
        # Store the values
        self.num_layers = num_layers
        self.num_nodes = num_nodes
        
        # Initialize the w and b & activation variables
        self.tf_w = dict()
        self.tf_b = dict()
        self.tb_w = dict()
        self.tb_b = dict()
        
        # Form the weights and biases
        with self.graph.as_default():
            with tf.name_scope("input") as scope:
                self.tf_x = tf.placeholder(tf.float32, shape=(self.batch_size, self.x_size))
                self.tf_y = tf.placeholder(tf.float32, shape=(self.batch_size, self.y_size))
                
            # If there are no layers, connect input to output directly
            if num_layers == 0:
                with tf.name_scope("output_layer") as scope:
                    self.tf_w[0] = tf.Variable(tf.truncated_normal([self.x_size, self.y_size], stddev=w_sd))
                    self.tf_b[0] = tf.Variable(tf.zeros([self.y_size]))
            else:
                # Add the first layer
                with tf.name_scope("hidden_layer_0") as scope:
                    self.tf_w[0] = tf.Variable(tf.truncated_normal([self.x_size, num_nodes[0]], stddev=w_sd))
                    self.tf_b[0] = tf.Variable(tf.zeros([num_nodes[0]]))

                # Add the intermediate layers
                for i in np.arange(1, num_layers):
                    with tf.name_scope("hidden_layer_"+str(i)) as scope:
                        self.tf_w[i] = tf.Variable(tf.truncated_normal([num_nodes[i-1], num_nodes[i]], stddev=w_sd))
                        self.tf_b[i] = tf.Variable(tf.zeros([num_nodes[i]]))

                # Connect to the output layer
                i = num_layers
                with tf.name_scope("output_layer") as scope:
                    self.tf_w[num_layers] = tf.Variable(tf.truncated_normal([num_nodes[i-1], self.y_size], stddev=w_sd))
                    self.tf_b[num_layers] = tf.Variable(tf.zeros([self.y_size]))
            
            # Add summary ops to collect data
            for i in np.arange(num_layers + 1):
                self.tb_w[i] = tf.histogram_summary("weights"+str(i), self.tf_w[i])
                self.tb_b[i] = tf.histogram_summary("biases"+str(i), self.tf_b[i])
            
            # Model the training, evaluation, cross validation and testing
            self.logits = self.__genModel(self.tf_x, self.tf_w, self.tf_b, self.num_layers, keep_prob)
            self.train = self.__genModel(self.tf_x, self.tf_w, self.tf_b, self.num_layers, 1.0)
            
            # Implement the loss function
            with tf.name_scope("cost_function") as scope:
                # Loss
                self.loss = tf.reduce_mean(tf.nn.l2_loss(self.logits - self.tf_y))

                # L2 regularization
                reg = 0
                for i in np.arange(self.num_layers + 1):
                    reg += tf.nn.l2_loss(self.tf_w[i]) + tf.nn.l2_loss(self.tf_b[i])
                reg *= reg_const

                # Add the regularization to the loss
                self.loss = tf.reduce_mean(self.loss + reg)

                # Create a summary to monitor the cost function
                tf.scalar_summary("cost_function", self.loss)

            # Create the global step variable
            self.gstep = 0

            # Implement the optimizer
            with tf.name_scope("back_prop") as scope:
                # Optimizer with a decaying learning rate
                self.lrate = tf.train.exponential_decay(init_lrate, self.gstep, 
                                                        lr_decay_step, lr_decay_rate, staircase=True)
                self.opt = tf.train.GradientDescentOptimizer(self.lrate).minimize(self.loss)

                # Create a summary to monitor the cost function
                tf.scalar_summary("learning_rate", self.lrate)
            
            # Merge all the summaries and start the summary writer
            self.summary = tf.merge_all_summaries()
    
    """
    Trains the model. This should be invoked only after the model is formed
    
    """
    def Train(self, num_steps, train_data, train_labels, cv_data, cv_labels, test_data, test_labels):
        with self.graph.as_default():
            # Add the test and cross validation data sets
            self.tf_cv = tf.constant(cv_data)
            self.tf_tst = tf.constant(test_data)

            # Set up the evaluations for the CV and test sets
            self.cv = self.__genModel(self.tf_cv, self.tf_w, self.tf_b, self.num_layers, 1.0)
            self.tst = self.__genModel(self.tf_tst, self.tf_w, self.tf_b, self.num_layers, 1.0)
            
        # Print the table format
        print('{:^20}'.format('Minibatch'),end="")
        print('{:^20}'.format('Time'),'{:^20}'.format('Loss'),end="")
        print('{:^20}'.format('Train accuracy'),'{:^20}'.format('Validation accuracy'))

        # Initialize the averages
        batch_avg_loss = 0.0
        batch_avg_acc = 0.0
        batch_step = 0

        # Start the session
        with tf.Session(graph=self.graph) as session:
            # Initialize the variables
            tf.initialize_all_variables().run()
            
            # Set the logs writer to the folder /tmp/tboard/logs/
            summary_writer = tf.train.SummaryWriter('/tmp/tboard/logs', graph=session.graph)

            # Note the starting time
            start_time = time.clock()
            
            # Loop through the steps
            for step in range(num_steps):
                # Pick an offset within the training data, which has been randomized.
                offset = (step * self.batch_size) % (train_labels.shape[0] - self.batch_size)
                
                # Generate a minibatch.
                data = train_data[offset:(offset + self.batch_size), :]
                labels = train_labels[offset:(offset + self.batch_size), :]
                
                # Run the session
                _, l, predictions, summary_str = session.run([self.opt, self.loss, self.train, self.summary],
                                                {self.tf_x : data, self.tf_y : labels})
                
                # Write logs for each iteration
                summary_writer.add_summary(summary_str, step)

                # Calculate the averages
                if step != 0:
                    batch_avg_loss = ((batch_avg_loss * (step - batch_step - 1)) + l) \
                                        / (step - batch_step)
                    batch_avg_acc = ((batch_avg_acc * (step - batch_step - 1)) + \
                                     accuracy(predictions, labels)) \
                                        / (step - batch_step)
                # At periodic intervals, print out the statistics
                if ((step % (int)(num_steps / 10) == 1) or (step == num_steps - 1)): 
                    # Print the values
                    print('{:^20}'.format(str(step)),end="")
                    print('{:^20}'.format(str(time.clock() - start_time)),end="")
                    print('{:^20}'.format(str(batch_avg_loss)),end="")
                    print('{:^20}'.format(str(batch_avg_acc)),end="")
                    print('{:^20}'.format(str(accuracy(self.cv.eval(), cv_labels))))
                    print('{:^20}'.format(str(predictions[0])),end="")
                    print('{:^20}'.format(str(labels[0])))

                    # Reset the values
                    start_time = time.clock()
                    batch_avg_loss = 0.0
                    batch_avg_acc = 0.0
                    batch_step = step
                
            # Print the test accuracy
            print("Test accuracy: %.1f%%" % accuracy(self.tst.eval(), test_labels))
        
    # Print utility
    def Print(self):
        for entry in self.__dict__:
            print (entry, self.__dict__[entry])

# Let's create a function to randomize the data set
def randomize(dataset, labels):
    permutation = np.random.permutation(labels.shape[0])
    shuffled_dataset = dataset[permutation,:]
    shuffled_labels = labels[permutation]
    return shuffled_dataset, shuffled_labels

"""
Gets image data
"""
def GetInput(csv_path, num_img, img_sz):
    
    # Create empty arrays to hold the images and labels
    data = np.ndarray(shape=(num_img, img_sz[0], img_sz[1]), dtype=np.float32)
    labels = np.ndarray(shape=(num_img, 1), dtype=np.float32)
    
    # Open the CSV file and parse it
    with open(os.path.join(csv_path,'interpolated.csv'), 'r') as csvfile:
        cnt = 0
        spamreader = csv.reader(csvfile, delimiter=',')
        for row in spamreader:
            # Skip the first row
            if row[0] != "index" and cnt < num_img:
                image = cv2.imread(os.path.join(csv_path,row[5]), 0)
                data[cnt,:,:] = cv2.resize(image, (0,0), fx=0.25, fy=0.25) 
                labels[cnt] = row[6]
                cnt = cnt + 1
    
    # Let's normalize the labels
    labels = labels - np.min(labels)
    labels = labels / np.max(labels)
    
    # Return the randomized labels
    return randomize(data, labels)
    
# Main function
def main():
    # Tuning parameters
    IMG_SZ = [120, 160]
    NUM_IMG = 17
    NUM_CV = 10
    NUM_TEST = 10
    BATCH_SZ = 8
    L_RATE = 0.001
    L_DECAY_STEP = 0.1
    L_DECAY_RATE = 0.8
    KEEP_PROB = 1.0
    REG_CONST = 0
    EPOCHS = 1000
    NN_NODES = np.array([100, 50])
    W_STDDEV = 0.003
    
    # Generated constants
    NUM_STEPS = (int)((NUM_IMG * EPOCHS) / BATCH_SZ)
    TOT_IMG = NUM_IMG + NUM_CV + NUM_TEST
    
    # Get the data
    data, labels = GetInput('/sharefolder/sdc-data/extract/', TOT_IMG, IMG_SZ)
    
    # Split the data into training, cross validation and testing sets
    TrainD = np.reshape(data[:NUM_IMG,:,:], (NUM_IMG, IMG_SZ[0] * IMG_SZ[1]))
    TrainL = np.reshape(labels[:NUM_IMG,:], (NUM_IMG, 1))
    
    ValidD = np.reshape(data[NUM_IMG:(NUM_IMG+NUM_CV),:,:], (NUM_CV, IMG_SZ[0] * IMG_SZ[1]))
    ValidL = np.reshape(labels[NUM_IMG:(NUM_IMG+NUM_CV),:], (NUM_CV, 1))
    
    TestD = np.reshape(data[(NUM_IMG+NUM_CV):,:,:], (NUM_TEST, IMG_SZ[0] * IMG_SZ[1]))
    TestL = np.reshape(labels[(NUM_IMG+NUM_CV):,:], (NUM_TEST, 1))
    
    # Initialize the NN
    myNet = NeuralNet(BATCH_SZ, TrainD.shape[1], TrainL.shape[1])

    # Define the model
    myNet.Model(len(NN_NODES), W_STDDEV, NN_NODES, KEEP_PROB, REG_CONST, L_RATE, 
                (L_DECAY_STEP * NUM_STEPS), L_DECAY_RATE)

    myNet.Train(NUM_STEPS, TrainD, TrainL, ValidD, ValidL, TestD, TestL)
    
    #myNet.Print()
    
# Script to execute the main
if __name__ == "__main__":
    main()



     Minibatch              Time                 Loss           Train accuracy    Validation accuracy 
         1          0.21798300000045856    0.585846722126      1.08244788647          1.33637       
   [ 0.18243623]       [ 0.59682423]    
        213          21.75861900000018     0.460385099405      0.935655421806         0.762129      
   [ 0.32074052]       [ 0.72076029]    
        425          21.779513999999836    0.265855848051      0.71992004972          0.819935      
   [ 0.40400708]       [ 0.00069777]    
        637          21.678541999999652    0.272418653206      0.729807570577         0.828552      
   [ 0.41142675]       [ 0.32968023]    
        849          21.94988900000044     0.273084905139      0.73074667212          0.829845      
   [ 0.41240123]       [ 0.34814748]    
        1061         22.031189999999697    0.271641644807      0.728752631624         0.830217      
   [ 0.41298494]       [ 0.20582323]    


KeyboardInterrupt: 